In [1]:
# from mat4py import loadmat
from scipy.io import loadmat
from pymatreader import read_mat
import pandas as pd
import numpy as np

In [2]:
subjects = read_mat("subjectData.mat")

In [3]:
class Data:
    def __init__(self, dataObj):
        self.subject1 = Subject(dataObj['subjectData']["preON"][0], dataObj['subjectData']["postON"][0])
#         self.subject2 = Subject(dataObj['subjectData']["preON"][1], dataObj['subjectData']["postON"][1])
        
class Subject:
    def __init__(self, preON, postON):
        self.fs = preON['header'][0]["fs"]
        self.eegLabels = preON['header'][0]["eegLabels"]
        #not sure how to use eegLocs, it's kinda gross looking
        self.eegLocs = preON['header'][0]["eegLocs"]
        
        self.pre_stim_runs = {}
        self.post_stim_runs = {}
        
        runs = ["run1", "run2", "run3"]
        for i, run in enumerate(runs):
            self.pre_stim_runs[run] = Run(preON['eeg'][i], preON['header'][i], self.fs)
            self.post_stim_runs[run] = Run(postON['eeg'][i], postON['header'][i], self.fs)
            
    def print_struct(self):
        print("Stored in Selected Subject::")
        print(f"fs: {self.fs}")
        print(f"EEG Labels: {self.eegLabels}")
        print(f"Number of runs pre-stimulation: {len(self.pre_stim_runs)}")
        print(f"Number of runs pre-stimulation: {len(self.pre_stim_runs)}")
        runs = ["run1", "run2", "run3"]
        print("Pre-Stimulation:")
        for i, run in enumerate(runs):
            print(f"    Data shape: {self.pre_stim_runs[run].raw_eeg.shape}, number of trials: {len(self.pre_stim_runs[run].trials)}")
        print("Post-Stimulation:")
        for i, run in enumerate(runs):
            print(f"    Data shape: {self.post_stim_runs[run].raw_eeg.shape}, number of trials: {len(self.pre_stim_runs[run].trials)}")
        
class Run:
    def __init__(self, eeg, header, fs):
        self.raw_eeg = np.array(eeg)
        
        trig_type = header["triggers"]["TYP"]
        trig_pos = header["triggers"]["POS"]
        
        trials = []
        for i, t_type in enumerate(trig_type):
            if t_type == 1000:
                trial = {}
                trial["rest_start"] = trig_pos[i]
            elif t_type == 786:
                trial["fixation_start"] = trig_pos[i]
            elif (t_type == 769):
                trial["cue_start"] = trig_pos[i]
                trial["hand"] = "right"
            elif (t_type == 770):
                trial["cue_start"] = trig_pos[i]
                trial["hand"] = "left"
            elif (t_type == 7691) or (t_type == 7701):
                trial["execution_start"] = trig_pos[i]
            elif (t_type == 7692) or (t_type == 7702):
                trial["end"] = trig_pos[i]
                seconds = (trig_pos[i] - trial["execution_start"]) / fs
                if seconds > 7.0:
                    trial["fail_type"] = "timeout"
                else:
                    trial["fail_type"] = "miss"
                trial["hit"] = False
                trials.append(trial)
            elif (t_type == 7693) or (t_type == 7703):
                trial["end"] = trig_pos[i]
                trial["hit"] = True
                trial["fail_type"] = "N/A"
                trials.append(trial)
        self.trials = trials
    
    
    


In [4]:
d = Data(subjects)
d.subject1.print_struct()

Stored in Selected Subject::
fs: 512
EEG Labels: ['FP1', 'FPZ', 'FP2', 'F7', 'F3', 'FZ', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 'M1', 'T7', 'C3', 'CZ', 'C4', 'T8', 'M2', 'CP5', 'CP1', 'CP2', 'CP6', 'P7', 'P3', 'PZ', 'P4', 'P8', 'POZ', 'O1', 'OZ', 'O2']
Number of runs pre-stimulation: 3
Number of runs pre-stimulation: 3
Pre-Stimulation:
    Data shape: (115200, 32), number of trials: 20
    Data shape: (115712, 32), number of trials: 20
    Data shape: (115712, 32), number of trials: 20
Post-Stimulation:
    Data shape: (138752, 32), number of trials: 20
    Data shape: (115200, 32), number of trials: 20
    Data shape: (107520, 32), number of trials: 20
